# Import packages

In [1]:
try:
    from requests_html import HTMLSession
except: 
    !pip install requests_html
    from requests_html import HTMLSession
    
try:
    import pandas as pd
    import numpy as np
except:
    !pip install pandas
    import pandas as pd
    import numpy as np

try:
    from Stockie.stockie import stockie
except:
    !pip install stockie
    !pip install seaborn
    from Stockie.stockie import stockie

try:
    import yfinance as yf
except:
    !pip install yfinance
    import yfinance as yf

try:
    from mplfinance.original_flavor import candlestick_ohlc
except:
    !pip install mplfinance
    from mplfinance.original_flavor import candlestick_ohlc
    
try:
    import plotly.graph_objects as go
    import plotly.subplots as ms
except:
    !pip install plotly
    import plotly.graph_objects as go
    import plotly.subplots as ms
    
try:
    from datetime import datetime
except:
    !pip install datetime
    from datetime import datetime
    
try:
    from ta import add_all_ta_features
    from ta.utils import dropna
except:
    !pip install ta 
    from ta import add_all_ta_features
    from ta.utils import dropna   
    
try:
    from scipy.signal import savgol_filter as smooth
except:
    !pip install scipy
    from scipy.signal import savgol_filter as smooth

try:
    import trendln
except:
    !pip install trendln
    import trendln

try:
    import investpy
except:
    !pip install investpy
    import investpy
    
try:
    import time
except:
    !pip install time
    import time

try:
    import quandl
except:
    !pip install quandl
    import quandl
    
try:
    import json
except:
    !pip install json
    import json
    
try:
    import requests
except:
    !pip install requests
    import requests

try:
    from bs4 import BeautifulSoup
except:
    !pip install bs4
    from bs4 import BeautifulSoup
    
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output


import warnings
warnings.filterwarnings('ignore')

/Users/vahid/myenv/lib/python3.9/site-packages/mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


# Define functions

In [2]:
# Get market info from INGSPRINTER
def market_info(market):
    market = market.replace('(', '')
    market = market.replace(')', '')
    market = market.replace(' ', '-')

    url = 'https://www.ingsprinters.nl/sprinters/' + market

    r = HTMLSession().get(url)

    value = r.html.find('span', clean=True)
    
    return float(value[4].text.strip().replace('.', '').replace(',', '.'))

def sprinter_list(market):
    
    url = 'https://www.ingsprinters.nl/zoeken?q=' + market

    r = HTMLSession().get(url)

    payload = {}
    data = r.html.find('a.fill-cell', clean=True)


    for item in data:
        if 'long'.capitalize() in item.text:
            payload[item.text] = re.sub(r'.*/NL', 'NL', item.attrs["href"])
        if 'short'.capitalize() in item.text:
            payload[item.text] = re.sub(r'.*/NL', 'NL', item.attrs["href"])

    return payload

In [3]:
def investing_com_histroy(pair_id, pair_id_for_news, pair_interval, Referer):
    url = 'https://www.investing.com/common/modules/js_instrument_chart/api/data.php?pair_id=' +\
    pair_id + '&pair_id_for_news=' + pair_id_for_news + '&chart_type=candlestick&pair_interval=' +\
    pair_interval + '&candle_count=120&events=yes&volume_series=yes&period='
    
    headers = {
        'User-Agent':
        'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': Referer
    }
    data = requests.get(url, headers=headers).json()

    df = pd.DataFrame(data['candles'],
                      columns=[
                          'date_time', 'open', 'high', 'low', 'close',
                          'volume', 'zero'
                      ])
    df['date_time'] = df.apply(
        lambda x: datetime.fromtimestamp(x['date_time'] // 1000), axis=1)
    return df



# Settings:

In [4]:
# Index List
watchlist = ['DAX']  #,'Dow Jones', 'S&P 500', 'AEX', 'CAC40', 'NASDAQ'
# Indexes tickeres per library
ticker_list = {
    'DAX': {
        'yfinance': '^GDAXI',
        'ingsprinter': 'DAX',
        'investpy': 'dax',
        'country': 'germany',
        'type': 'index',
        'pair_id': '172',
        'pair_id_for_news': '172',
        'Referer': 'https://www.investing.com/indices/germany-30'
        
    },
    'NASDAQ': {
        'yfinance': '^IXIC',
        'ingsprinter': 'NASDAQ-100',
        'investpy': 'nasdaq',
        'country': 'united states',
        'type': 'index',
        'pair_id': '20',
        'pair_id_for_news': '20',
        'Referer': 'https://www.investing.com/indices/nq-100'
    }
}

first_round_interval = ['1mo', '1wk', '1d']
second_round_inverval = ['1h', '30m', '15m', '5m']

Periods = {
    '3mo': '5y',
    '1mo': '5y',
    '1wk': '2y',
    '1d': '1y',
    '1h': 'ytd',
    '30m': '3mo',
    '15m': '1mo',
    '5m': '5d'
}

interval_yfinance_to_investpy = {
    '1mo': 'monthly',
    '1wk': 'weekly',
    '1d': 'daily',
    '1h': '1hour',
    '5h': '5hours',
    '30m': '30mins',
    '15m': '15mins',
    '5m': '5mins'
}


interval_investing_com={
    '1h': '3600',
    '5h': '18000',
    '30m': '1800',
    '15m': '900',
    '5m': '300'
} 

# Analysis

## Analysis of Monthly, weekly, daily data of indexes

In [6]:
# load Monthly, weekly, daily data of indexes
Analysis_data = {}
for i in watchlist:
    Analysis_data[i] = {}
    for j in first_round_interval:
        Analysis_data[i][j] = yf.download(ticker_list[i]['yfinance'],
                                                  period=Periods[j],
                                                  interval=j)
        Analysis_data[i][j + '_ta_features'] = add_all_ta_features(
            Analysis_data[i][j],
            open="Open",
            high="High",
            low="Low",
            close="Close",
            volume="Volume",
            fillna=True)
        Analysis_data[i][j + '_pivot_points'] = investpy.pivot_points(
            name=ticker_list[i]['investpy'],
            country=ticker_list[i]['country'],
            product_type=ticker_list[i]['type'],
            interval=interval_yfinance_to_investpy[j])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
for i in watchlist:
    for j in second_round_inverval:
        Analysis_data[i][j] = investing_com_histroy(
            ticker_list[i]['pair_id'], ticker_list[i]['pair_id_for_news'],
            interval_investing_com[j], ticker_list[i]['Referer']).rename(
                columns={
                    'open': "Open",
                    'high': "High",
                    'low': "Low",
                    'close': "Close",
                    'volume': "Volume"
                }) 
        Analysis_data[i][j + '_ta_features'] = add_all_ta_features(
            Analysis_data[i][j],
            open="Open",
            high="High",
            low="Low",
            close="Close",
            volume="Volume",
            fillna=True)
        Analysis_data[i][j + '_pivot_points'] = investpy.pivot_points(
            name=ticker_list[i]['investpy'],
            country=ticker_list[i]['country'],
            product_type=ticker_list[i]['type'],
            interval=interval_yfinance_to_investpy[j])

# Effectiveness of indicators - backtesting

In [8]:
#Check effectiveness of ta indicators 
# 1. corr with shift (1), shift (2), roling_avg(5) 
# 2. regression (auto selection) / timeseries / ML
# 3. youtube approach

In [9]:
Analysis_data['DAX'].keys()

dict_keys(['1mo', '1mo_ta_features', '1mo_pivot_points', '1wk', '1wk_ta_features', '1wk_pivot_points', '1d', '1d_ta_features', '1d_pivot_points', '1h', '1h_ta_features', '1h_pivot_points', '30m', '30m_ta_features', '30m_pivot_points', '15m', '15m_ta_features', '15m_pivot_points', '5m', '5m_ta_features', '5m_pivot_points'])